# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [21]:
# Import necessary packages
import pandas as pd
import sqlite3 

In [22]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
cur = conn.cursor()

In [23]:
ls

CONTRIBUTING.md      README.md            index.ipynb
LICENSE.md           contact_list.pickle  school.sqlite


## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [24]:
cur.execute("""CREATE TABLE contactInfo(userId INTEGER PRIMARY KEY,
                                        firstName TEXT,
                                        lastName TEXT,
                                        role TEXT,
                                        telephone INTEGER,
                                        street TEXT,
                                        city TEXT,
                                        state TEXT,
                                        zipcode INTEGER);
            """)

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [25]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [26]:
contacts

[{'firstName': 'Christine',
  'lastName': 'Holden',
  'role': 'staff',
  'telephone ': 2035687697,
  'street': '1672 Whitman Court',
  'city': 'Stamford',
  'state': 'CT',
  'zipcode ': '06995'},
 {'firstName': 'Christopher',
  'lastName': 'Warren',
  'role': 'student',
  'telephone ': 2175150957,
  'street': '1935 University Hill Road',
  'city': 'Champaign',
  'state': 'IL',
  'zipcode ': '61938'},
 {'firstName': 'Linda',
  'lastName': 'Jacobson',
  'role': 'staff',
  'telephone ': 4049446441,
  'street': '479 Musgrave Street',
  'city': 'Atlanta',
  'state': 'GA',
  'zipcode ': '30303'},
 {'firstName': 'Andrew',
  'lastName': 'Stepp',
  'role': 'student',
  'telephone ': 7866419252,
  'street': '2981 Lamberts Branch Road',
  'city': 'Hialeah',
  'state': 'Fl',
  'zipcode ': '33012'},
 {'firstName': 'Jane',
  'lastName': 'Evans',
  'role': 'student',
  'telephone ': 3259909290,
  'street': '1461 Briarhill Lane',
  'city': 'Abilene',
  'state': 'TX',
  'zipcode ': '79602'},
 {'firstNa

In [27]:
contacts[0].keys()

dict_keys(['firstName', 'lastName', 'role', 'telephone ', 'street', 'city', 'state', 'zipcode '])

In [28]:
contacts[0].values()

dict_values(['Christine', 'Holden', 'staff', 2035687697, '1672 Whitman Court', 'Stamford', 'CT', '06995'])

In [29]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    firstName = contact['firstName']
    lastName = contact['lastName']
    role = contact['role']
    telephone = contact['telephone ']
    street = contact['street']
    city = contact['city']
    state = contact['state']
    zipcode = contact['zipcode ']
    cur.execute("""INSERT INTO contactInfo(firstName, lastName, role, telephone, street, city, state, zipcode)
                                           VALUES('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');""".format(firstName, lastName, role, telephone, street, city, state, zipcode))
    


**Query the Table to Ensure it is populated**

In [35]:
cur.execute("""SELECT * 
               FROM contactInfo;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,userId,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [15]:
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [16]:
# Create the grades table
cur.execute("""CREATE TABLE grades(
                   userId INTEGER NOT NULL,
                   courseId INTEGER NOT NULL,
                   grade TEXT,
                   PRIMARY KEY(userId, courseId)
                   );
            """)

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [19]:
# Find the duplicate entry
df.duplicated() # not being pulled up b/c Jane Evans actually has 2 different userId's so the row isn't identical

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
dtype: bool

In [45]:
cur.execute("""SELECT firstName, lastName, telephone, COUNT(*) AS count
               FROM contactInfo
               GROUP BY firstName, lastName, telephone
               HAVING count > 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [40]:
# Delete the duplicate entry
cur.execute(""" DELETE FROM contactInfo WHERE firstName = 'Jane';""")

In [43]:
# Check that the duplicate entry was removed
cur.execute("""SELECT firstName, lastName, telephone, COUNT(*) AS count
               FROM contactInfo
               GROUP BY firstName, lastName, telephone
               HAVING count > 1;""").fetchall()

[]

## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [46]:
cur.execute("""SELECT * 
               FROM contactInfo;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,userId,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639


In [54]:
# Update Ed's address
cur.execute("""UPDATE contactInfo
               SET street = '2910 Simpson Avenue',
                   city = 'York',
                   state = 'PA',
                   zipcode = 17403
               WHERE lastName = 'Lyman';""")

In [55]:
# Query the database to ensure the change was made
cur.execute("""SELECT * 
               FROM contactInfo;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,userId,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
5,8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [56]:
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 